In [1]:
import os
import random
import yaml
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

from latency_estimator import LatencyEstimator
from tqdm.notebook import tqdm
from random_search import proxylessnas_random
from search_spaces import load_proxylessnas

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
IMG_SIZE = (224, 224)
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    return tf.image.resize(image, size=IMG_SIZE), label

In [4]:
ds, info = tfds.load('caltech101', as_supervised=True, with_info=True)

In [5]:
raw_train = ds['train']
raw_test = ds['test']

In [6]:
BATCH_SIZE = 128

raw_valid, raw_test = raw_test.take(3000), raw_test.skip(3000)
train, valid, test = raw_train.map(preprocess_image), raw_valid.map(preprocess_image), raw_test.map(preprocess_image)
train_batches, valid_batches, test_batches = train.shuffle(1024).batch(BATCH_SIZE), valid.batch(BATCH_SIZE), test.batch(BATCH_SIZE)

In [ ]:
reps = proxylessnas_random(300)

model_acc = []

for rep in tqdm(reps):
    tf.keras.backend.clear_session()
    
    model = load_proxylessnas()(weights=None, classes=info.features['label'].num_classes, arch_kernel=rep)
    
    lr_schedule = tf.keras.experimental.CosineDecay(0.001, decay_steps=10000)
    model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr_schedule, rho=0.9, momentum=0.9, epsilon=0.1), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    model.fit(train_batches, epochs=90, verbose=0)
    
    _, accuracy = model.evaluate(valid_batches)
    _, test_accuracy = model.evaluate(test_batches)
    model_acc.append({'arch': rep, 'val_acc': accuracy, 'test_acc': test_accuracy})

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
np.save('outputs/model_acc_caltech.npy', model_acc)